In [1]:
#import dependencies
import psycopg2
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [2]:
emissions = pd.read_csv('emissions.csv')
emissions

,iso_code,country,year,temperature(C),population,gdp,co2,co2_per_capita,gdp_per_capita,co2_per_gdp,share_global_co2,co2_growth_prct,co2_growth_abs,cumulative_co2,share_global_cumulative_co2,global_co2_to_pop,global_impact,emission_ratio
0,AFG,Afghanistan,1950,11.619864,7752000.0,1.949480e+10,0.084,0.011,2514.808999,0.004,0.001,475.000,0.070,0.099,0.000,0.012900,Low,Low
1,AFG,Afghanistan,1951,12.647118,7840000.0,2.006385e+10,0.092,0.012,2559.164343,0.005,0.001,8.696,0.007,0.191,0.000,0.012755,Low,Low
2,AFG,Afghanistan,1953,13.272074,8040000.0,2.201546e+10,0.106,0.013,2738.241719,0.005,0.002,16.000,0.015,0.388,0.000,0.024876,Low,Low
3,AFG,Afghanistan,1955,13.275183,8271000.0,2.292989e+10,0.154,0.019,2772.323695,0.007,0.002,44.828,0.048,0.649,0.000,0.024181,Low,Low
4,AFG,Afghanistan,1956,12.785808,8399000.0,2.395993e+10,0.183,0.022,2852.712089,0.008,0.002,19.048,0.029,0.832,0.000,0.023812,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11268,ZWE,Zimbabwe,2012,21.910075,13115000.0,2.048226e+10,7.659,0.584,1561.743118,0.374,0.022,-19.213,-1.821,690.720,0.049,0.167747,Low,High
11269,ZWE,Zimbabwe,2013,21.624350,13350000.0,2.374258e+10,11.617,0.870,1778.470621,0.489,0.033,51.675,3.958,702.337,0.049,0.247191,Low,High
11270,ZWE,Zimbabwe,2014,21.710483,13587000.0,2.474828e+10,11.973,0.881,1821.467867,0.484,0.034,3.065,0.356,714.309,0.048,0.250239,Low,High
11271,ZWE,Zimbabwe,2015,22.327625,13815000.0,2.503057e+10,12.170,0.881,1811.840028,0.486,0.035,1.653,0.198,726.480,0.048,0.253348,Low,High


## Analyize High and Low Emission Ratio Countries
The countries in this dataset have been split into two groups, High and Low.  This represents the ratio between each country's GDP, and its CO2 emissions.  A High ranking means that country has a ratio over .3, indicating that their emissions outstrip their economic output.  In other words, the economies of these countries produce more emissions than economic benefit to it's citizens.  An analysis of these two groups can help determine if it's possible for a country to both grow it's economy and keep emissions below this threshold.  The changes over time for key metrics will be examined for each country to show trends.

The countries have also been evaluated based on their pollution contribution relative to the rest of the world.  Each country's Share Global CO2 has been divided by it's population (multiplied by 100000000 for readability), for a new column Global CO2 to Pop.  This number is then labelled as high, medium, or low.  A ratio over 4 will rate as 'high', between 4 and 1 will rate as 'medium', and less than 1 will rate as low.  4 was chosen as the cutoff because it encompasses many countries with economies anchored in oil and gas and thus are known to have higher emissions. Comparing share of global co2 to population allows for an evaluation of each country's impact on the world relative to how many world citizens that country supports.

### Emissions for Each Country

In [19]:
# separate into high and low emission groups
high_emission = emissions.loc[emissions['emission_ratio'] == 'High']
low_emission = emissions.loc[emissions['emission_ratio'] == 'Low']

In [4]:
high_emission['country'].unique().tolist()

['Albania',
 'Algeria',
 'Angola',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Barbados',
 'Belarus',
 'Belgium',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Cape Verde',
 'Chile',
 'China',
 'Colombia',
 'Republic of the Congo ',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'Equatorial Guinea',
 'Estonia',
 'Finland',
 'France',
 'Gabon',
 'Georgia',
 'Germany',
 'Greece',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kuwait',
 'Kyrgyzstan',
 'Laos',
 'Latvia',
 'Lebanon',
 'Lesotho',
 'Liberia',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Malta',
 'Mauritania',
 'Mexico',
 'Moldova',
 'Mongolia',
 'Montenegro',
 'Morocco',
 'Mozambique',
 'Netherlands',
 'New Zealand',
 'North Korea',
 'North Macedonia',
 'Norway',
 'Oman',
 'Pana

In [5]:
low_emission['country'].unique().tolist()

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Argentina',
 'Armenia',
 'Austria',
 'Azerbaijan',
 'Bangladesh',
 'Barbados',
 'Belgium',
 'Benin',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Republic of the Congo ',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Haiti',
 'Honduras',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kenya',
 'Kuwait',
 'Kyrgyzstan',
 'Laos',
 'Latvia',
 'Lebanon',
 'Liberia',
 'Libya',

By separating the dataframe into low and high emission lists, and examining the countries, it is clear that many countries appear as both high and low, depending on the year.  Now we can begin to determine how a country's Emission Ratio has changed over time.  First a range of the years in the dataset must be established for each country, as it varies depending on the availability of data.

In [6]:
# find the first and last year in the dataset for each country
date_range = emissions.groupby('country').agg({'year':['min','max']})['year']
date_range

,min,max
country,,
Afghanistan,1950,2016
Albania,1950,2016
Algeria,1970,2016
Angola,1975,2016
Argentina,1901,2016
...,...,...
Venezuela,1914,2016
Vietnam,1913,2016
Yemen,1951,2016


A for loop is written that will loop through each country in the dataset, and determine the GDP, Emission Ratio, CO2 produced, and the Temperature for the first and last year available.  A calculation is then done to determine the change over time for each of these indicators.  This will separate countries into groups of emissions ratios over time as Low to Low, High to Low, Low to High, and High to High.  It will also show how much GDP has increased or decreased, the change in CO2 output, and how the temperature has fluctuated over the time span given for each country.  This will allow us to find the countries that have done the best and the worst over time.

In [7]:
# create a list of countries in the dataset
countries = emissions['country'].unique().tolist()

# creat empty dictionaries to hold the new values
stay_low = {}
improved = {}
got_worse = {}
stay_high = {}

#loop through the list of countries
for country in countries:
    
    #create a variable to hold the first and last year in the dataset for each country
    first = date_range.loc[country,'min']
    last = date_range.loc[country,'max']
    
    #create a variable to hold the value for gdp for the first and last year for each country
    gdp_start = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['gdp'].values[0]
    gdp_end = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['gdp'].values[0]
    
    #create a variable to hold the 'high' or 'low' ranking for each country's emissions in the first and last year
    start_ratio = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['emission_ratio'].values[0]
    end_ratio = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['emission_ratio'].values[0]
    
    # create variable to hold co2 for the first and last year for each country
    start_co2 = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['co2'].values[0]
    end_co2 = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['co2'].values[0]
    
    # create variable to hold temperature for the first and last year for each country
    start_temp = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['temperature(C)'].values[0]
    end_temp = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['temperature(C)'].values[0]
    
    
    # calculate the change in gdp, co2, and temp for countries that went from LOW TO LOW 
    if (start_ratio == 'Low') & (end_ratio == 'Low'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        stay_low[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
       
    # calculate the change in gdp, co2, and temp for countries that changed from HIGH TO LOW 
    elif (start_ratio == 'High') & (end_ratio == 'Low'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        improved[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
        
    # calculate the change in gdp, co2, and temp for countries that changed from LOW TO HIGH 
    elif (start_ratio == 'Low') & (end_ratio == 'High'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        got_worse[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
    
    # calculate the change in gdp, co2, and temp for countries that changed from HIGH TO HIGH 
    elif (start_ratio == 'High') & (end_ratio == 'High'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        stay_high[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
        
        
# convert calulations to dataframe
stay_low_df = pd.DataFrame.from_dict(stay_low, orient = 'index')
improved_df = pd.DataFrame.from_dict(improved, orient = 'index')
got_worse_df = pd.DataFrame.from_dict(got_worse, orient = 'index')
stay_high_df = pd.DataFrame.from_dict(stay_high, orient = 'index')

The result is four dataframes which show the countries that have Emissions Ratios that have stayed low, improved, gotten worse, and stayed high over time. Each dataframe shows the changes to GDP, CO2 and Temperature.  The range of years is also included for context.

In [8]:
#clean up index on stay_low
stay_low_df = stay_low_df.reset_index()
stay_low= stay_low_df.drop(['index'], axis = 1)
stay_low

,country,gdp_change,co2_change,temperature_change,year_start,year_end
0,Afghanistan,4.480263e+10,6.661,2.893701,1950,2016
1,Albania,3.253803e+10,4.199,0.294388,1950,2016
2,Algeria,4.457547e+11,133.780,1.402083,1970,2016
3,Angola,1.288973e+11,29.702,1.195092,1975,2016
4,Argentina,7.959566e+11,188.438,-0.258302,1901,2016
...,...,...,...,...,...,...
78,Tunisia,1.123060e+11,27.789,1.517540,1950,2016
79,Turkey,1.542490e+12,399.203,1.841345,1913,2016
80,Uganda,7.088775e+10,4.666,1.474542,1951,2016
81,Uruguay,6.059736e+10,6.460,0.121688,1933,2016


In [9]:
#clean up index on improved
improved_df = improved_df.reset_index()
improved= improved_df.drop(['index'], axis = 1)

In [10]:
#clean up index on got_worse
got_worse_df = got_worse_df.reset_index()
got_worse = got_worse_df.drop(['index'], axis = 1)

In [11]:
#clean up index on stay_high
stay_high_df = stay_high_df.reset_index()
stay_high = stay_high_df.drop(['index'], axis = 1)

The stay_ low and improved dataframes are then merged, as all these countries have earned the distinction of having done a good job by maintaining or improving their emissions/gdp ratio.

In [12]:
#merge two dataframes into one
good_job = pd.concat([stay_low, improved])
good_job

,country,gdp_change,co2_change,temperature_change,year_start,year_end
0,Afghanistan,4.480263e+10,6.661,2.893701,1950,2016
1,Albania,3.253803e+10,4.199,0.294388,1950,2016
2,Algeria,4.457547e+11,133.780,1.402083,1970,2016
3,Angola,1.288973e+11,29.702,1.195092,1975,2016
4,Argentina,7.959566e+11,188.438,-0.258302,1901,2016
...,...,...,...,...,...,...
28,Sweden,4.216962e+11,34.451,1.191047,1901,2016
29,Switzerland,5.087460e+11,34.008,2.379357,1901,2016
30,Tajikistan,7.167521e+09,-1.409,1.225386,1973,2016
31,United Kingdom,2.349000e+12,-11.591,1.172031,1901,2016


The got_worse and stay_high dataframes are then merged, as all these countries have earned the distinction of having done a bad job by showing a deteriorating ratio or by never improving at all.

In [13]:
#merge two dataframes into one
bad_job = pd.concat([got_worse, stay_high])
bad_job

,country,gdp_change,co2_change,temperature_change,year_start,year_end
0,Barbados,1.991235e+09,1.212,1.388825,1950,2016
1,Bolivia,6.400739e+10,20.682,0.027000,1929,2016
2,Bulgaria,1.188880e+11,43.987,1.870271,1921,2016
3,China,1.655600e+13,9524.671,1.737695,1911,2016
4,Cuba,7.340974e+10,28.969,1.552817,1942,2015
5,India,7.230000e+12,2379.159,1.005800,1901,2016
6,Iran,1.236554e+12,647.207,2.286862,1913,2016
7,Iraq,5.294403e+11,192.712,2.092709,1950,2016
8,Jamaica,1.814499e+10,7.621,1.434400,1951,2016
9,Kuwait,1.750000e+11,82.262,2.235508,1974,2016


The bad_job dataframe is ordered to show the countries that have performed the worst in terms of CO2 emissions over time.  This is done by keeping the countries with a positive CO2 change over time and putting them in order from worst to least worst.  This will indicate the worst polluters.

In [14]:
polluters = bad_job.loc[bad_job['co2_change'] > 1 ].sort_values(by= 'co2_change', ascending=False)
polluters     

,country,gdp_change,co2_change,temperature_change,year_start,year_end
3,China,1.655600e+13,9524.671,1.737695,1911,2016
20,United States,1.666900e+13,4569.086,2.339558,1901,2016
5,India,7.230000e+12,2379.159,1.005800,1901,2016
12,Russia,2.338000e+12,732.443,2.880001,1960,2016
6,Iran,1.236554e+12,647.207,2.286862,1913,2016
17,South Korea,1.843363e+12,617.948,2.231083,1911,2016
13,Saudi Arabia,1.483510e+12,560.610,1.901842,1950,2016
4,Canada,1.542802e+12,540.188,2.241518,1901,2016
0,Australia,1.048156e+12,399.593,0.905133,1901,2016
15,South Africa,5.762403e+11,399.012,1.493617,1950,2016


Sorting by the countries with the least gdp growth, that also increased co2 emissions, will show those who have done the least for its citizens and the least for the environment.

In [15]:
the_worst = bad_job.loc[(bad_job['co2_change'] > 1)].sort_values(by= 'gdp_change', ascending=True)
the_worst

,country,gdp_change,co2_change,temperature_change,year_start,year_end
0,Barbados,1.991235e+09,1.212,1.388825,1950,2016
12,Liberia,2.832623e+09,1.314,0.829717,1951,2016
8,Jamaica,1.814499e+10,7.621,1.434400,1951,2016
22,Zimbabwe,1.953991e+10,7.635,1.052292,1950,2016
16,Trinidad and Tobago,3.260620e+10,37.700,1.356325,1950,2016
9,Mongolia,3.311642e+10,25.031,1.498112,1951,2016
3,Bosnia and Herzegovina,4.010089e+10,19.201,0.900939,1952,2016
15,North Korea,4.123232e+10,24.298,2.653862,1913,2015
10,Laos,4.298193e+10,14.202,1.588592,1956,2016
13,Libya,4.978613e+10,42.670,1.235208,1951,2016


The good_jobs dataframe is filtered to only include countries that have increased GDP AND decreased CO2 output.  This is determined by only including countries with a positive GDP change over time, and a negative CO2 change over time.  This will give us the countries that have shown it's possible to grow an economy and decrease emissions over time.

In [16]:
#filter for countries who both increased gdp and decreased co2
bravo = good_job.loc[(good_job['co2_change'] < 1) & (good_job['gdp_change'] > 1)].sort_values(by= 'co2_change', ascending=True)
bravo

,country,gdp_change,co2_change,temperature_change,year_start,year_end
26,Slovakia,7.011266e+10,-24.226,2.302522,1985,2016
18,Moldova,6.694083e+09,-15.846,1.766712,1973,2016
31,United Kingdom,2.349000e+12,-11.591,1.172031,1901,2016
7,Georgia,7.914037e+09,-11.183,1.545461,1973,2016
15,Lithuania,4.136707e+10,-10.612,1.098658,1973,2016
0,Armenia,1.040713e+10,-8.678,1.675355,1973,2016
13,Kyrgyzstan,1.241572e+10,-6.418,1.304525,1973,2016
14,Latvia,1.943851e+10,-5.559,1.081291,1973,2016
2,Azerbaijan,9.651863e+10,-2.328,1.387227,1973,2016
30,Tajikistan,7.167521e+09,-1.409,1.225386,1973,2016


In [21]:
countrylist = emissions['country'].unique().tolist()
len(countrylist)

160

In [22]:
the_good_ones = bravo['country'].unique().tolist()
len(the_good_ones)

24

### Country Emissions Compared to Global Emissions

In [18]:
# separate into high and low emission groups
high_impact = emissions.loc[emissions['global_impact'] == 'High']
med_impact = emissions.loc[emissions['global_impact'] == 'Medium']
low_impact = emissions.loc[emissions['global_impact'] == 'Low']

In [22]:
# create a list of countries in the dataset
countries = emissions['country'].unique().tolist()

# creat empty dictionaries to hold the new values
stay_low = {}
improved_alot = {}
improved_some = {}
little_worse = {}
way_worse = {}
stay_high = {}

#loop through the list of countries
for country in countries:
    
    #create a variable to hold the first and last year in the dataset for each country
    first = date_range.loc[country,'min']
    last = date_range.loc[country,'max']
    
    #create a variable to hold the value for gdp for the first and last year for each country
    gdp_start = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['gdp'].values[0]
    gdp_end = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['gdp'].values[0]
    
    #create a variable to hold the ranking for each country's global impact in the first and last year
    start_impact = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['global_impact'].values[0]
    end_impact = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['global_impact'].values[0]
    
    # create variable to hold co2 for the first and last year for each country
    start_co2 = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['co2'].values[0]
    end_co2 = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['co2'].values[0]
    
    # create variable to hold temperature for the first and last year for each country
    start_temp = emissions[(emissions['country'] == country) & (emissions['year'] == first)]['temperature(C)'].values[0]
    end_temp = emissions[(emissions['country'] == country) & (emissions['year'] == last)]['temperature(C)'].values[0]
    
    
    # calculate the change in gdp, co2, and temp for countries that went from LOW TO LOW 
    if (start_impact == 'Low') & (end_impact == 'Low'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        stay_low[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
       
    # calculate the change in gdp, co2, and temp for countries that changed from HIGH TO LOW 
    elif (start_impact == 'High') & (end_impact == 'Low'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        improved_alot[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
    # calculate the change in gdp, co2, and temp for countries that changed from HIGH TO MEDIUM
    elif (start_impact == 'High') & (end_impact == 'Medium'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        improved_some[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
        
    # calculate the change in gdp, co2, and temp for countries that changed from LOW TO MEDIUM
    elif (start_impact == 'Medium') & (end_impact == 'High'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        little_worse[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
        
    # calculate the change in gdp, co2, and temp for countries that changed from LOW TO HIGH 
    elif (start_impact == 'Low') & (end_impact == 'High'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        way_worse[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
    
    # calculate the change in gdp, co2, and temp for countries that changed from HIGH TO HIGH 
    elif (start_impact == 'High') & (end_impact == 'High'):
        gdp_change = gdp_end - gdp_start
        co2_change = end_co2 - start_co2
        temp_change = end_temp - start_temp
        stay_high[country] = {'country' : country,
                            'gdp_change': gdp_change, 
                            'co2_change' : co2_change,
                            'temperature_change' : temp_change,
                            'year_start' : first,
                            'year_end' : last}
        
        
# convert calulations to dataframe
impact_stay_low_df = pd.DataFrame.from_dict(stay_low, orient = 'index')
impact_improved_alot_df = pd.DataFrame.from_dict(improved_alot, orient = 'index')
impact_improved_some_df = pd.DataFrame.from_dict(improved_some, orient = 'index')
impact_little_worse_df = pd.DataFrame.from_dict(little_worse, orient = 'index')
impact_way_worse_df = pd.DataFrame.from_dict(way_worse, orient = 'index')
impact_stay_high_df = pd.DataFrame.from_dict(stay_high, orient = 'index')

In [27]:
#clean up index on stay_low
stay_low_df = impact_stay_low_df.reset_index()
low_impact = stay_low_df.drop(['index'], axis = 1)
low_impact

,country,gdp_change,co2_change,temperature_change,year_start,year_end
0,Afghanistan,4.480263e+10,6.661,2.893701,1950,2016
1,Albania,3.253803e+10,4.199,0.294388,1950,2016
2,Angola,1.288973e+11,29.702,1.195092,1975,2016
3,Bangladesh,4.822873e+11,74.864,0.467233,1950,2016
4,Benin,2.080477e+10,6.637,0.954667,1959,2016
...,...,...,...,...,...,...
73,Tunisia,1.123060e+11,27.789,1.517540,1950,2016
74,Uganda,7.088775e+10,4.666,1.474542,1951,2016
75,Uruguay,6.059736e+10,6.460,0.121688,1933,2016
76,Vietnam,5.559898e+11,184.080,1.309325,1913,2016


In [28]:
#clean up index on improved_alot
improved_alot_df = impact_improved_alot_df.reset_index()
improved_alot = improved_alot_df.drop(['index'], axis = 1)

In [29]:
#clean up index on impact_improved_some
improved_some_df = impact_improved_some_df.reset_index()
improved_some = improved_some_df.drop(['index'], axis = 1)

In [30]:
#clean up index on little worse
little_worse_df = impact_little_worse_df.reset_index()
little_worse = little_worse_df.drop(['index'], axis = 1)

In [31]:
#clean up index on way_worse
way_worse_df = impact_way_worse_df.reset_index()
way_worse = way_worse_df.drop(['index'], axis = 1)

In [32]:
#clean up index on high impact
stay_high_impact = impact_stay_high_df.reset_index()
high_impact = stay_high_impact.drop(['index'], axis = 1)